In [ ]:
import requests
import numpy as np
from pprint import pprint

In [2]:
url = "http://127.0.0.1:8800"

## ENV

In [5]:
response = requests.get(f"{url}/env")
pprint(response.json())

{'objects': {'object_bowl_0': {'id': 209,
                               'ori': [-2.6379038041381816e-06,
                                       2.1143758985875394e-06,
                                       -8.092278815619685e-06],
                               'pos': [-2.2000012857675637,
                                       -0.04999866043529781,
                                       0.9624646425706653]},
             'object_fork_0': {'id': 208,
                               'ori': [8.248408994755436e-17,
                                       0.0,
                                       6.4380296469890066e-18],
                               'pos': [-0.2999999999988103,
                                       -0.6499999999995241,
                                       0.9397844891595659]}},
 'robot': {'arm_joints': [-0.011929967646029572,
                          -1.0322412031875856,
                          0.05096959347384768,
                          -2.2595210535106935,
 

In [47]:
response = requests.get(f"{url}/env_entire")
all = response.json()

print(all.keys())

dict_keys(['timestamp', 'objects', 'robot', 'fixtures', 'groups', 'objects_by_group', 'ungrouped_objects'])


In [48]:
pprint(all["objects"])

{'object_bowl_0': {'group': 'counter_1_left_left_group',
                   'id': 209,
                   'ori': [-1.630873900090902e-06,
                           -2.0885891127697676e-07,
                           -9.039139958439375e-06],
                   'pos': [-2.2000015178437344,
                           -0.04999858520059568,
                           0.9624646548846464]},
 'object_fork_0': {'group': 'island_left_group',
                   'id': 208,
                   'ori': [5.150529194420065e-17, 0.0, 6.4485896982982085e-18],
                   'pos': [-0.2999999999988103,
                           -0.6499999999995241,
                           0.9397844891595659]}}


In [ ]:
pprint(all["groups"]) 

{'cab_1_left_group': {'boundary': [[-2.6000040606587937,
                                    -0.750000848526225,
                                    1.3900000000000001],
                                   [-2.152002145654715,
                                    -0.24999927875304634,
                                    2.31]],
                      'fixtures': ['cab_1_left_group_main',
                                   'cab_1_left_group_hingedoor',
                                   'cab_1_left_group_door_main',
                                   'cab_1_left_group_door_handle_main']},
 'cab_2_left_group': {'boundary': [[-2.600013060658794,
                                    1.750000151473775,
                                    1.3900000000000001],
                                   [-2.1520093598402705,
                                    2.250001721246954,
                                    2.31]],
                      'fixtures': ['cab_2_left_group_main',
                        

In [52]:
# pprint(all["objects"])

# pprint(all["robot"]

# pprint(all["fixtures"])

pprint(all["objects_by_group"])

{'cab_1_left_group': [],
 'cab_2_left_group': [],
 'cab_corner_3_left_group': [],
 'cab_corner_4_left_group': [],
 'cab_main_main_group': [],
 'counter_1_left_left_group': ['object_bowl_0'],
 'counter_1_right_main_group': [],
 'counter_corner_left_group': [],
 'counter_corner_main_main_group': [],
 'counter_main_main_group': [],
 'fridge_cab_main_group': [],
 'fridge_housing_main_group': [],
 'fridge_main_group': [],
 'hood_main_group': [],
 'island_left_group': ['object_fork_0'],
 'microwave_left_group': [],
 'shelves_main_group': [],
 'sink_left_group': [],
 'stack_1_left_group': [],
 'stack_1_main_group': [],
 'stack_2_left_group': [],
 'stack_2_main_group': [],
 'stack_3_left_group': [],
 'stack_3_main_group': [],
 'stovetop_main_group': [],
 'window_group': []}


In [53]:
pprint(all["ungrouped_objects"])

[]


## Mobile

In [34]:
def convert_list_to_str(lst):
    # if lis is [1.0, 2.0, 3.0], return "[1.0, 2.0, 3.0]"
    return "[" + ", ".join([str(x) for x in lst]) + "]"


def set_mobile_position(target_pos):
    payload = {
        "action": {
            "type": "run_code",
            "payload": {
                "code": """
pos = get_mobile_joint_position()
RESULT = {
    "mobile_joint_position": pos
}
"""
            },
        }
    }
    response = requests.post(f"{url}/send_action", json=payload)
    current_pos = response.json()["result"]["mobile_joint_position"]
    current_pos = np.array(current_pos)
    print("Current mobile joint position:", current_pos)  # x, y, theta

    if isinstance(target_pos, list):
        target_postion = np.array(target_pos)

    delta_pos = target_pos + current_pos
    print("Delta to target position:", delta_pos)

    delta_pos_str = convert_list_to_str(delta_pos)

    payload = {
        "action": {
            "type": "run_code",
            "payload": {
                "code": f"""
result = set_mobile_target_joint({delta_pos_str})
RESULT = result
"""
            },
        }
    }
    response = requests.post(f"{url}/send_action", json=payload)


set_mobile_position([0, -0.2, 0])

Current mobile joint position: [-1.0783174  -0.91714628  6.27257565]
Delta to target position: [-1.0783174  -1.11714628  6.27257565]


In [41]:
def get_object_postion(object_name: str | None = None):
    payload = {
        "action": {
            "type": "run_code",
            "payload": {
                "code": """
objects = get_object_positions()
RESULT = {
    "objects": objects
}
"""
            },
        }
    }

    response = requests.post(f"{url}/send_action", json=payload)
    objects = response.json()["result"]["objects"]
    # if object_name in objects, get key

    if object_name is not None:
        for key in objects.keys():
            if object_name in key:
                object_key = key
                break

        obj_pos = objects[object_key]["pos"]
        return obj_pos
    else:
        return objects


obj_pos = get_object_postion("bowl")
print(obj_pos)

[-2.2000014372390657, -0.04999869255576094, 0.9624647411979161]


In [42]:
def set_mobile_target(target_pos):
    target_pos_str = convert_list_to_str(target_pos)
    payload = {
        "action": {
            "type": "run_code",
            "payload": {
                "code": f"""
result = set_mobile_target_joint({target_pos_str})
RESULT = result
"""
            },
        }
    }
    response = requests.post(f"{url}/send_action", json=payload)
    response.json()


set_mobile_target(obj_pos)

In [ ]:
def convert_list_to_str(lst):
    # if lis is [1.0, 2.0, 3.0], return "[1.0, 2.0, 3.0]"
    return "[" + ", ".join([str(x) for x in lst]) + "]"


def move_to_position(pos):
    pos_str = convert_list_to_str(pos)

    payload = {
        "action": {
            "type": "run_code",
            "payload": {
                "code": f"""
result = set_mobile_target_joint({pos_str})
RESULT = result
"""
            },
        }
    }
    response = requests.post(f"{url}/send_action", json=payload)
    return response.json()


move_to_position([-1, -1, 0.0])

In [ ]:
obj_pos = get_object_postion("fork")
print(obj_pos)